# IBM Data Science Certificate - Capstone Project

## City of Southampton Land Usage

### Fernando Apolinar


## 1. Importing required Libraries

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='0aca10de-b0c5-4780-8f59-1c05b1d75142', project_access_token='p-f511bdff9e20f1f1b774e283940d89fd358c8b0f')
pc = project.project_context


In [2]:
import itertools # To iterate over dataframe to geolocate
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import scipy.optimize as opt
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm 
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm.auto import tqdm
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01


## 1.2.  Import data from Watson Studio Cloud and showing SouthEast London data set.

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0a1ee5a3b7134e84afb70ccbe488700c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Nvm6KPl6uO6lwAdpfA-aK0roavUe5DMGnf4hKq54iKLn',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0a1ee5a3b7134e84afb70ccbe488700c.get_object(Bucket='capstoneproject-donotdelete-pr-3aka0rp0llp7nt',Key='open_postcode_geo_south_east.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

col_names=["postcode", "status", "usertype", "easting", "northing", "positional_quiality_indicator", "country", "latitude", "longitude", "postcode_no_space", "postcode_fixed_width_seven", "postcode_fixed_width_eight", "postcode_area", "postcode_district", "postcode_sector", "outcode", "incode", "local_authority_code", "local_authority_name"]
    
df_data_ESTLOND = pd.read_csv(body, names=col_names)
df_data_ESTLOND.head()



,postcode,status,usertype,easting,northing,positional_quiality_indicator,country,latitude,longitude,postcode_no_space,postcode_fixed_width_seven,postcode_fixed_width_eight,postcode_area,postcode_district,postcode_sector,outcode,incode,local_authority_code,local_authority_name
0,ME1 1AA,live,large,574570,168212,1,England,51.386104,0.507423,ME11AA,ME1 1AA,ME1 1AA,ME,ME1,ME1 1,ME1,1AA,E06000035,Medway
1,ME1 1AB,terminated,large,574571,168217,1,England,51.386148,0.507440,ME11AB,ME1 1AB,ME1 1AB,ME,ME1,ME1 1,ME1,1AB,E06000035,Medway
2,ME1 1AD,live,small,574369,167805,1,England,51.382509,0.504337,ME11AD,ME1 1AD,ME1 1AD,ME,ME1,ME1 1,ME1,1AD,E06000035,Medway
3,ME1 1AE,live,small,574472,168193,1,England,51.385963,0.506007,ME11AE,ME1 1AE,ME1 1AE,ME,ME1,ME1 1,ME1,1AE,E06000035,Medway
4,ME1 1AF,live,small,574462,168221,1,England,51.386218,0.505877,ME11AF,ME1 1AF,ME1 1AF,ME,ME1,ME1 1,ME1,1AF,E06000035,Medway


## 3. Filtering Southampton Area ZIPCODES

In [17]:
df_SOTON = df_data_ESTLOND[(df_data_ESTLOND.postcode_area == "SO") & (df_data_ESTLOND.status == "live")]
df_SOTON.head()

,postcode,status,usertype,easting,northing,positional_quiality_indicator,country,latitude,longitude,postcode_no_space,postcode_fixed_width_seven,postcode_fixed_width_eight,postcode_area,postcode_district,postcode_sector,outcode,incode,local_authority_code,local_authority_name
85080,SO14 0AA,live,small,442293,112125,1,England,50.907044,-1.399853,SO140AA,SO140AA,SO14 0AA,SO,SO14,SO14 0,SO14,0AA,E06000045,Southampton
85081,SO14 0AB,live,large,442383,112134,1,England,50.907128,-1.398581,SO140AB,SO140AB,SO14 0AB,SO,SO14,SO14 0,SO14,0AB,E06000045,Southampton
85083,SO14 0AE,live,small,442352,112315,1,England,50.908757,-1.399001,SO140AE,SO140AE,SO14 0AE,SO,SO14,SO14 0,SO14,0AE,E06000045,Southampton
85084,SO14 0AF,live,small,443030,112868,1,England,50.913671,-1.389284,SO140AF,SO140AF,SO14 0AF,SO,SO14,SO14 0,SO14,0AF,E06000045,Southampton
85085,SO14 0AG,live,small,442220,112890,1,England,50.913937,-1.400812,SO140AG,SO140AG,SO14 0AG,SO,SO14,SO14 0,SO14,0AG,E06000045,Southampton


## 4. Preparing geo data coordinates - sampling to 1000 rows to not overcome API call limit

In [5]:
df_new_SOTON = df_SOTON[["postcode_no_space", "latitude", "longitude"]]
df_new_SOTON["geom"] = df_SOTON["latitude"].map(str) + "," + df_SOTON["longitude"].map(str)
df_rand_SOTON = df_new_SOTON.sample(n=1000, random_state=1)
#df_new_SOTON["geom"][0]
df_rand_SOTON.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,postcode_no_space,latitude,longitude,geom
88561,SO163ZG,50.943673,-1.402195,"50.943673,-1.402195"
199314,SO452PQ,50.829176,-1.390921,"50.829176000000004,-1.3909209999999999"
89113,SO166QH,50.930476,-1.429927,"50.930476,-1.429927"
213318,SO230JF,51.058879,-1.303841,"51.058879,-1.303841"
90347,SO172NZ,50.925921,-1.384537,"50.925921,-1.3845370000000001"


In [7]:
locator = Nominatim(user_agent="ny_explorer", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
df_rand_SOTON.shape

(1000, 4)

## 5. Reverse Geocoding
### As data set is already geocoded, user reverse method to get address meta data for each location.

In [9]:
tqdm.pandas()
df_rand_SOTON["address"] = df_rand_SOTON["geom"].progress_apply(rgeocode)
df_rand_SOTON.head()

,postcode_no_space,latitude,longitude,geom,address
88561,SO163ZG,50.943673,-1.402195,"50.943673,-1.402195","(F Block, Glen Eyre Drive, Glen Eyre, Southamp..."
199314,SO452PQ,50.829176,-1.390921,"50.829176000000004,-1.3909209999999999","(Depedene Close, Fawley, Holbury, New Forest, ..."
89113,SO166QH,50.930476,-1.429927,"50.930476,-1.429927","(32, Dale Road, Upper Shirley, Southampton, So..."
213318,SO230JF,51.058879,-1.303841,"51.058879,-1.303841","(Quarry Road, Saint Giles Hill, Winchester, Ha..."
90347,SO172NZ,50.925921,-1.384537,"50.925921,-1.3845370000000001","(St Denys Court, Flat 1;2;3;4;5, Kent Road, Sa..."


In [10]:
df_rand_SOTON.shape

(1000, 5)

## Saving not to geocode each time.

In [15]:
from project_lib import Project
project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON.csv",data = df_rand_SOTON.to_csv(index=False))

{'file_name': 'SOTON.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': 'c71c5046-7a6d-4cb1-b85c-3ca35037b9e0'}

In [20]:
df_rand_SOTON.dtypes

postcode_no_space     object
latitude             float64
longitude            float64
geom                  object
address               object
dtype: object

## 6. Extracting Metadata from address column

In [22]:
df_rand_SOTON['address'] = df_rand_SOTON['address'].apply(str) 
df_rand_SOTON.dtypes

postcode_no_space     object
latitude             float64
longitude            float64
geom                  object
address               object
dtype: object

In [25]:
df_rand_SOTON['address1'], df_rand_SOTON['street'],df_rand_SOTON['district'], df_rand_SOTON['city'],df_rand_SOTON['county'] = df_rand_SOTON['address'].str.split(',', 4).str
df_rand_SOTON.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


,postcode_no_space,latitude,longitude,geom,address,address1,street,district,city,county
88561,SO163ZG,50.943673,-1.402195,"50.943673,-1.402195","F Block, Glen Eyre Drive, Glen Eyre, Southampt...",F Block,Glen Eyre Drive,Glen Eyre,Southampton,"South East, England, SO16 3NQ, United Kingdom"
199314,SO452PQ,50.829176,-1.390921,"50.829176000000004,-1.3909209999999999","Depedene Close, Fawley, Holbury, New Forest, H...",Depedene Close,Fawley,Holbury,New Forest,"Hampshire, South East, England, SO45 2PQ, Uni..."
89113,SO166QH,50.930476,-1.429927,"50.930476,-1.429927","32, Dale Road, Upper Shirley, Southampton, Sou...",32,Dale Road,Upper Shirley,Southampton,"South East, England, SO16 6QH, United Kingdom"
213318,SO230JF,51.058879,-1.303841,"51.058879,-1.303841","Quarry Road, Saint Giles Hill, Winchester, Ham...",Quarry Road,Saint Giles Hill,Winchester,Hampshire,"South East, England, SO23 0HR, United Kingdom"
90347,SO172NZ,50.925921,-1.384537,"50.925921,-1.3845370000000001","St Denys Court, Flat 1;2;3;4;5, Kent Road, Sai...",St Denys Court,Flat 1;2;3;4;5,Kent Road,Saint Denys,"Southampton, South East, England, SO17 2NZ, U..."


## Filtering relevant Columns

In [26]:
df_rand_SOTON.drop(columns=['address1', 'street','county'])

,postcode_no_space,latitude,longitude,geom,address,district,city
88561,SO163ZG,50.943673,-1.402195,"50.943673,-1.402195","F Block, Glen Eyre Drive, Glen Eyre, Southampt...",Glen Eyre,Southampton
199314,SO452PQ,50.829176,-1.390921,"50.829176000000004,-1.3909209999999999","Depedene Close, Fawley, Holbury, New Forest, H...",Holbury,New Forest
89113,SO166QH,50.930476,-1.429927,"50.930476,-1.429927","32, Dale Road, Upper Shirley, Southampton, Sou...",Upper Shirley,Southampton
213318,SO230JF,51.058879,-1.303841,"51.058879,-1.303841","Quarry Road, Saint Giles Hill, Winchester, Ham...",Winchester,Hampshire
90347,SO172NZ,50.925921,-1.384537,"50.925921,-1.3845370000000001","St Denys Court, Flat 1;2;3;4;5, Kent Road, Sai...",Kent Road,Saint Denys
...,...,...,...,...,...,...,...
90563,SO181DZ,50.922578,-1.374878,"50.922578,-1.374878","Cobden Court, Bitterne Park, Southampton, Sout...",Southampton,South East
213819,SO237BZ,51.067374,-1.318656,"51.067374,-1.318656","Hayward Guitars, Stockbridge Road, Fulflood, W...",Fulflood,Winchester
91701,SO189SS,50.907683,-1.406979,"50.907683,-1.406979","Southampton City Council, Civic Centre, Civic ...",Civic Centre Road,The Polygon
171821,SO509QW,50.973651,-1.365637,"50.973651000000004,-1.365637","Eastleigh, Hampshire, South East, England, SO5...",South East,England


## Checking NA

In [27]:
df_rand_SOTON['address'].isna().sum()

0

## Saving for future use

In [80]:
print(df_rand_SOTON['district'].isna().sum())
print(df_rand_SOTON['city'].isna().sum())
from project_lib import Project
project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON_Clean_v2.csv",data = df_rand_SOTON.to_csv(index=False))

0
0


{'file_name': 'SOTON_Clean_v2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': '5ef93d7b-b6c3-4a0b-9ac2-22ac5848e151'}

In [19]:
body = client_0a1ee5a3b7134e84afb70ccbe488700c.get_object(Bucket='capstoneproject-donotdelete-pr-3aka0rp0llp7nt',Key='SOTON_Clean.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_rand_SOTON = pd.read_csv(body)
df_rand_SOTON.head()

,postcode_no_space,latitude,longitude,geom,address,address1,street,district,city,county
0,SO163ZG,50.943673,-1.402195,"50.943673,-1.402195","F Block, Glen Eyre Drive, Glen Eyre, Southampt...",F Block,Glen Eyre Drive,Glen Eyre,Southampton,"South East, England, SO16 3NQ, United Kingdom"
1,SO452PQ,50.829176,-1.390921,"50.829176000000004,-1.3909209999999999","Depedene Close, Fawley, Holbury, New Forest, H...",Depedene Close,Fawley,Holbury,New Forest,"Hampshire, South East, England, SO45 2PQ, Uni..."
2,SO166QH,50.930476,-1.429927,"50.930476,-1.429927","32, Dale Road, Upper Shirley, Southampton, Sou...",32,Dale Road,Upper Shirley,Southampton,"South East, England, SO16 6QH, United Kingdom"
3,SO230JF,51.058879,-1.303841,"51.058879,-1.303841","Quarry Road, Saint Giles Hill, Winchester, Ham...",Quarry Road,Saint Giles Hill,Winchester,Hampshire,"South East, England, SO23 0HR, United Kingdom"
4,SO172NZ,50.925921,-1.384537,"50.925921,-1.3845370000000001","St Denys Court, Flat 1;2;3;4;5, Kent Road, Sai...",St Denys Court,Flat 1;2;3;4;5,Kent Road,Saint Denys,"Southampton, South East, England, SO17 2NZ, U..."


## 7. locating in Map

In [20]:
# create map of Southampton using latitude and longitude values
latitude = "50.909698"
longitude = "-1.404351"
map_SOTON = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, city in zip(df_rand_SOTON['latitude'], df_rand_SOTON['longitude'], df_rand_SOTON['district'], df_rand_SOTON['city']):
    label = '{}, {}'.format(district, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SOTON)  
    
map_SOTON

In [13]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/AlucarD980/Coursera_Capstone/blob/main/Results/SOTON_data.png?raw=true")

In [23]:
body = client_0a1ee5a3b7134e84afb70ccbe488700c.get_object(Bucket='capstoneproject-donotdelete-pr-3aka0rp0llp7nt',Key='data_asset/SOTON_Clean_v2_csv_shaped_eck2u2vucquzgbkdsrl7ky15f')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_city_SOTON = pd.read_csv(body)
df_city_SOTON.head()


,postcode_no_space,latitude,longitude,district,city
0,SO166QH,50.930476,-1.429927,Upper Shirley,Southampton
1,SO211PP,51.115074,-1.254709,Itchen Valley,Winchester
2,SO516GF,50.955399,-1.536292,Copythorne,Southampton
3,SO169AD,50.918852,-1.458948,Millbrook,Southampton
4,SO185DU,50.920047,-1.342917,Harefield,Southampton


## 8. Calling Foursquare API to get venue data

In [34]:
CLIENT_ID = '1J40KTD0N22UEKII0YY2K4FLNAU4OWJVSDXXVPW4ATRXE2TG' # your Foursquare ID
CLIENT_SECRET = 'WY4UG5XK4SXC0KMULGSQI3NGVSVKNY5V5PCFAJXQDF1BDNPU' # your Foursquare Secret
ACCESS_TOKEN = 'II00AEZZ11L3TFSGSBTTZRB3ATUNFZZP3LUNIUO4XWXIERYQ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
df_fsqr = df_city_SOTON.copy()

Your credentails:
CLIENT_ID: 1J40KTD0N22UEKII0YY2K4FLNAU4OWJVSDXXVPW4ATRXE2TG
CLIENT_SECRET:WY4UG5XK4SXC0KMULGSQI3NGVSVKNY5V5PCFAJXQDF1BDNPU


In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 800 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1J40KTD0N22UEKII0YY2K4FLNAU4OWJVSDXXVPW4ATRXE2TG&client_secret=WY4UG5XK4SXC0KMULGSQI3NGVSVKNY5V5PCFAJXQDF1BDNPU&v=20180604&ll=50.909698,-1.404351&radius=800&limit=100'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603803884fdbb60cdec6de19'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'City Centre',
  'headerFullLocation': 'City Centre, Southampton',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 86,
  'suggestedBounds': {'ne': {'lat': 50.916898007200004,
    'lng': -1.392953605192828},
   'sw': {'lat': 50.902497992799994, 'lng': -1.4157483948071719}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f511895e4b00ad2ab0f52da',
       'name': 'Mango Thai Tapas Bar',
       'location': {'address': '180-182 Above Bar Street',
        'crossStreet': 'Cultural Quarter',
        'lat': 50.90873978717777,
        'lng': -1.4047925038110263,
 

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Mango Thai Tapas Bar,Thai Restaurant,50.908740,-1.404793
1,Halladays Tearooms,Tea Room,50.911482,-1.405862
2,The Stable,Pizza Place,50.907612,-1.404548
3,Guildhall Square,Event Space,50.907989,-1.405240
4,Belgium & Blues,Beer Bar,50.908802,-1.404705


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare.


## 9. Venues per district

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
southampton_venues = getNearbyVenues(names=df_city_SOTON['district'],
                                   latitudes=df_city_SOTON['latitude'],
                                   longitudes=df_city_SOTON['longitude']
                                  )

 Upper Shirley
 Itchen Valley
 Copythorne
 Millbrook
 Harefield
 Glen Eyre
 Droxford
 Hightown
 Redbridge
 Warsash
 Cheriton
 Maybush
 Aldermoor
 Sholing
 Newtown-Nicholstown
 Bishop’s Waltham
 Saint Denys
 Northam
 Twyford
 Townhill Park
 Owslebury
 Portswood
 New Alresford
 Olivers Battery
 Badger Farm
 Curbridge
 Teg Down
 Weston
 St Mary's
 Springvale
 Stanmore
 Lower Upham
 Freemantle
 Bitterne Park
 Soberton Heath
 Swanwick
 Woolston
 Hedge End
 Littleton
 Marchwood
 Fulflood
 Winnall
 Midanbury
 Bar End
 Hollybrook
 Shedfield
 Chilcomb
 Bramdean
 Bassett
 Bitterne
 Headbourne Worthy
 Lordswood
 Shirley
 Mansbridge
 Kings Worthy
 Swaythling
 Micheldever
 Bevois Town
 Wimpson
 Highfield
 Lords Hill
 Newtown
 The Polygon
 Banister Park
 Otterbourne
 Meonstoke
 Waltham Chase
 Hyde
 Itchen
 Shirley Warren
 Weeke
 Coxford
 Chilworth
 Thornhill
 Exton
 Swanmore


In [45]:
print(southampton_venues.shape)
southampton_venues.head()

(489, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Upper Shirley,50.930476,-1.429927,PureGym,50.930051,-1.425689,Gym / Fitness Center
1,Upper Shirley,50.930476,-1.429927,St James' Park,50.928935,-1.430935,Park
2,Upper Shirley,50.930476,-1.429927,M&S Simply Food,50.933514,-1.434392,Grocery Store
3,Upper Shirley,50.930476,-1.429927,The Malvern,50.931952,-1.423835,Pub
4,Upper Shirley,50.930476,-1.429927,Wickes Extra,50.930278,-1.424703,Hardware Store


In [62]:
print('There are {} uniques categories.'.format(len(southampton_venues['Venue Category'].unique())))

project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON_venues2.csv",data = southampton_venues.to_csv(index=False))

There are 125 uniques categories.


{'file_name': 'SOTON_venues2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': '12c9199c-c08a-44c6-b2e3-4c4e90b24876'}

## 10. One hot coding

In [49]:
# one hot encoding
southampton_onehot = pd.get_dummies(southampton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
southampton_onehot['Neighborhood'] = southampton_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [southampton_onehot.columns[-1]] + list(southampton_onehot.columns[:-1])
southampton_onehot = southampton_onehot[fixed_columns]

southampton_onehot.head()

,Neighborhood,Alternative Healer,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,...,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Wine Shop,Yoga Studio
0,Upper Shirley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Upper Shirley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Upper Shirley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Upper Shirley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Upper Shirley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
southampton_onehot.shape

(489, 126)

## 10. Most venue frequency

In [53]:
southampton_grouped = southampton_onehot.groupby('Neighborhood').mean().reset_index()
southampton_grouped

,Neighborhood,Alternative Healer,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,...,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Wine Shop,Yoga Studio
0,Aldermoor,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Badger Farm,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Banister Park,0.000000,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bar End,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bassett,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Weeke,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,Weston,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,Wimpson,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,Winnall,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Saving for future use

In [61]:
southampton_grouped.shape
project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON_freq_venues2.csv",data = southampton_grouped.to_csv(index=False))

{'file_name': 'SOTON_freq_venues2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': '2bb140ff-f061-4a30-bb66-746adc4d4976'}

## 11. Calculating top venues per district

In [56]:
num_top_venues = 5

for hood in southampton_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = southampton_grouped[southampton_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Aldermoor----
                venue   freq
0  Alternative Healer  0.333
1   Fish & Chips Shop  0.333
2   Mobile Phone Shop  0.333
3       Movie Theater  0.000
4         Pizza Place  0.000


---- Badger Farm----
               venue  freq
0         Playground   0.2
1  Other Repair Shop   0.2
2        Event Space   0.2
3        Coffee Shop   0.2
4        Supermarket   0.2


---- Banister Park----
               venue   freq
0                Pub  0.167
1                Bar  0.167
2           Beer Bar  0.083
3  Convenience Store  0.083
4          Nightclub  0.083


---- Bar End----
           venue   freq
0            Pub  0.250
1     Restaurant  0.250
2  Historic Site  0.167
3         Garden  0.083
4    Bus Station  0.083


---- Bassett----
              venue  freq
0               Bar   0.2
1              Hill   0.2
2  Business Service   0.2
3          Bus Stop   0.2
4            Hostel   0.2


---- Bevois Town----
          venue   freq
0         Hotel  0.222
1           Bar  0.111

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = southampton_grouped['Neighborhood']

for ind in np.arange(southampton_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(southampton_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldermoor,Alternative Healer,Mobile Phone Shop,Fish & Chips Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market
1,Badger Farm,Supermarket,Playground,Coffee Shop,Other Repair Shop,Event Space,Yoga Studio,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,Banister Park,Bar,Pub,Pizza Place,Café,Grocery Store,Asian Restaurant,Convenience Store,Hotel,Beer Bar,Nightclub
3,Bar End,Pub,Restaurant,Historic Site,Pizza Place,Garden,Bus Station,River,Yoga Studio,Electronics Store,Fast Food Restaurant
4,Bassett,Hill,Bus Stop,Business Service,Hostel,Bar,Fast Food Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Farmers Market


In [63]:
project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON_neighborhoods_venues2.csv",data = neighborhoods_venues_sorted.to_csv(index=False))

{'file_name': 'SOTON_neighborhoods_venues2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': '7d78b5ef-8f94-4606-bb77-897b26873d04'}

## 12. Clustering to group areas by venues type density using KMEANS

In [65]:
# set number of clusters
kclusters = 5

southampton_grouped_clustering = southampton_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(southampton_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 1, 3], dtype=int32)

In [68]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Aldermoor,Alternative Healer,Mobile Phone Shop,Fish & Chips Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market
1,3,Badger Farm,Supermarket,Playground,Coffee Shop,Other Repair Shop,Event Space,Yoga Studio,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,3,Banister Park,Bar,Pub,Pizza Place,Café,Grocery Store,Asian Restaurant,Convenience Store,Hotel,Beer Bar,Nightclub
3,3,Bar End,Pub,Restaurant,Historic Site,Pizza Place,Garden,Bus Station,River,Yoga Studio,Electronics Store,Fast Food Restaurant
4,3,Bassett,Hill,Bus Stop,Business Service,Hostel,Bar,Fast Food Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Farmers Market


In [70]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
southampton_venues2 = southampton_venues.drop(columns=['Venue'])

southampton_merged = southampton_venues2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
southampton_merged = southampton_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

southampton_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Upper Shirley,50.930476,-1.429927,50.930051,-1.425689,Gym / Fitness Center,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
1,Upper Shirley,50.930476,-1.429927,50.928935,-1.430935,Park,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
2,Upper Shirley,50.930476,-1.429927,50.933514,-1.434392,Grocery Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
3,Upper Shirley,50.930476,-1.429927,50.931952,-1.423835,Pub,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
4,Upper Shirley,50.930476,-1.429927,50.930278,-1.424703,Hardware Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store


## Saving....

In [75]:
project = Project(project_id="0aca10de-b0c5-4780-8f59-1c05b1d75142",project_access_token="p-f511bdff9e20f1f1b774e283940d89fd358c8b0f")
project.save_data(file_name = "SOTON_neighborhoods_geocluster_venues2.csv",data = southampton_merged.to_csv(index=False))

{'file_name': 'SOTON_neighborhoods_geocluster_venues2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-3aka0rp0llp7nt',
 'asset_id': 'd2b9c65e-932f-4e2a-99ff-9d201e15887b'}

In [4]:

body = client_0a1ee5a3b7134e84afb70ccbe488700c.get_object(Bucket='capstoneproject-donotdelete-pr-3aka0rp0llp7nt',Key='SOTON_neighborhoods_geocluster_venues2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

southampton_merged = pd.read_csv(body)
southampton_merged.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Upper Shirley,50.930476,-1.429927,50.930051,-1.425689,Gym / Fitness Center,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
1,Upper Shirley,50.930476,-1.429927,50.928935,-1.430935,Park,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
2,Upper Shirley,50.930476,-1.429927,50.933514,-1.434392,Grocery Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
3,Upper Shirley,50.930476,-1.429927,50.931952,-1.423835,Pub,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
4,Upper Shirley,50.930476,-1.429927,50.930278,-1.424703,Hardware Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store


## 13. Showing clusters on MAP

In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(southampton_merged['Venue Latitude'], southampton_merged['Venue Longitude'], southampton_merged['Neighborhood'], southampton_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [14]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/AlucarD980/Coursera_Capstone/blob/main/Results/SOTON_Clusters.png?raw=true")

## 14. Checking at each Cluster - Cluster: 0

In [77]:
southampton_merged.loc[southampton_merged['Cluster Labels'] == 0, southampton_merged.columns[[1] + list(range(5, southampton_merged.shape[1]))]]

,Neighborhood Latitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,50.851449,Pub,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
44,50.851449,Grocery Store,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
45,50.851449,Convenience Store,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
46,50.851449,Pub,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
47,50.851449,Indian Restaurant,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
48,50.851449,Deli / Bodega,0,Pub,Convenience Store,Deli / Bodega,Grocery Store,Indian Restaurant,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
195,50.891486,Grocery Store,0,Pub,Grocery Store,Chinese Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
196,50.891486,Chinese Restaurant,0,Pub,Grocery Store,Chinese Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
197,50.891486,Pub,0,Pub,Grocery Store,Chinese Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
198,50.891486,Pub,0,Pub,Grocery Store,Chinese Restaurant,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store


## 14.1 Checking at each Cluster - Cluster: 1

In [78]:
southampton_merged.loc[southampton_merged['Cluster Labels'] == 1, southampton_merged.columns[[1] + list(range(5, southampton_merged.shape[1]))]]

,Neighborhood Latitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,50.923853,Playground,1,Playground,Park,Food & Drink Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
42,50.923853,Park,1,Playground,Park,Food & Drink Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
170,50.919705,Playground,1,Playground,French Restaurant,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market
289,51.094469,Park,1,Yoga Studio,Park,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market
290,51.094469,Yoga Studio,1,Yoga Studio,Park,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market
307,50.927045,Park,1,Park,Construction & Landscaping,Pub,Yoga Studio,Creperie,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
308,50.927045,Park,1,Park,Construction & Landscaping,Pub,Yoga Studio,Creperie,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
309,50.927045,Pub,1,Park,Construction & Landscaping,Pub,Yoga Studio,Creperie,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
310,50.927045,Construction & Landscaping,1,Park,Construction & Landscaping,Pub,Yoga Studio,Creperie,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store


## 14.2 Checking at each Cluster - Cluster: 2

In [79]:
southampton_merged.loc[southampton_merged['Cluster Labels'] == 2, southampton_merged.columns[[1] + list(range(5, southampton_merged.shape[1]))]]

,Neighborhood Latitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,50.930522,Grocery Store,2,Athletics & Sports,Grocery Store,Flea Market,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
50,50.930522,Flea Market,2,Athletics & Sports,Grocery Store,Flea Market,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
51,50.930522,Athletics & Sports,2,Athletics & Sports,Grocery Store,Flea Market,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
55,50.907337,Sandwich Place,2,Playground,Sandwich Place,Grocery Store,Bar,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
56,50.907337,Grocery Store,2,Playground,Sandwich Place,Grocery Store,Bar,Event Space,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Electronics Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,50.926991,Supermarket,2,Supermarket,Furniture / Home Store,Park,Grocery Store,Sandwich Place,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space
478,50.937240,Grocery Store,2,Bus Stop,Coffee Shop,Grocery Store,Stationery Store,Yoga Studio,Farmers Market,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
479,50.937240,Stationery Store,2,Bus Stop,Coffee Shop,Grocery Store,Stationery Store,Yoga Studio,Farmers Market,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
480,50.937240,Coffee Shop,2,Bus Stop,Coffee Shop,Grocery Store,Stationery Store,Yoga Studio,Farmers Market,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


## 14.3 Checking at each Cluster - Cluster: 3

In [80]:
southampton_merged.loc[southampton_merged['Cluster Labels'] == 3, southampton_merged.columns[[1] + list(range(5, southampton_merged.shape[1]))]]

,Neighborhood Latitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,50.930476,Gym / Fitness Center,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
1,50.930476,Park,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
2,50.930476,Grocery Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
3,50.930476,Pub,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
4,50.930476,Hardware Store,3,Gym / Fitness Center,Hardware Store,Grocery Store,Park,Pet Store,Coffee Shop,Pub,Café,Furniture / Home Store,Stationery Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,50.906003,Coffee Shop,3,Furniture / Home Store,Coffee Shop,Bakery,Fast Food Restaurant,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
484,50.906003,Furniture / Home Store,3,Furniture / Home Store,Coffee Shop,Bakery,Fast Food Restaurant,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
485,50.906003,Fast Food Restaurant,3,Furniture / Home Store,Coffee Shop,Bakery,Fast Food Restaurant,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store
486,50.906003,Bakery,3,Furniture / Home Store,Coffee Shop,Bakery,Fast Food Restaurant,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store


## 14.4 Checking at each Cluster - Cluster: 4

In [81]:
southampton_merged.loc[southampton_merged['Cluster Labels'] == 4, southampton_merged.columns[[1] + list(range(5, southampton_merged.shape[1]))]]

,Neighborhood Latitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,50.963860,Gastropub,4,Clothing Store,Gastropub,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
36,50.963860,Clothing Store,4,Clothing Store,Gastropub,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
137,50.906715,Gastropub,4,Gastropub,Yoga Studio,French Restaurant,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
262,51.061512,Gastropub,4,Chinese Restaurant,Gastropub,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
263,51.061512,Chinese Restaurant,4,Chinese Restaurant,Gastropub,Yoga Studio,French Restaurant,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
487,50.982548,Gastropub,4,Gastropub,Yoga Studio,French Restaurant,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Donut Shop,Electronics Store,Event Space
